# 环境

In [4]:
import requests
import json
import time
import pprint

headers = {
    'Content-Type': 'application/json'
}

# Swagger: http://localhost:8080/swagger-ui/index.html
HOST='localhost'
URL_PREFIX = 'http://{host}:8080'.format(host=HOST)

# Requests
# https://docs.python-requests.org/en/latest/api
def print_request(pr):
    body = pr.body
    if body != None and len(body) > 1000:
        body = str(body[:100]) + '...'
    print("""=== >
{method} {url}
{headers}
{body}""".format(method=pr.method, url=pr.url, headers=pr.headers, body=body))

def print_response(r):
    print("====== Response\n", r.status_code)
    print_request(r.request)
    print("=== <")
    print(r.headers)
    if 'content-type' in r.headers and  'application/json' in r.headers['content-type']:
        pprint.pprint(r.json())
    else:
        if len(r.content) > 1000:
            print(r.content[:100], '...')
        else:
            print(r.content)

# Metadata

## GET /api/metadata/workflow
Retrieves all workflow definition along with blueprint

In [10]:
url = URL_PREFIX + '/api/metadata/workflow'
params = {
}
print(url)
pprint.pprint(headers)
pprint.pprint(params)

r = requests.get(url=url, headers=headers, params=params)
print_response(r)

http://localhost:8080/api/metadata/workflow
{'Content-Type': 'application/json'}
{}
====== Response
 200
=== >
GET http://localhost:8080/api/metadata/workflow
{'User-Agent': 'python-requests/2.31.0', 'Accept-Encoding': 'gzip, deflate', 'Accept': '*/*', 'Connection': 'keep-alive', 'Content-Type': 'application/json'}
None
=== <
{'Content-Type': 'application/json', 'Transfer-Encoding': 'chunked', 'Date': 'Mon, 08 Jul 2024 01:45:11 GMT', 'Keep-Alive': 'timeout=60', 'Connection': 'keep-alive'}
[{'accessPolicy': {},
  'createTime': 1720403106567,
  'description': 'First Sample Workflow',
  'inputParameters': [],
  'inputTemplate': {},
  'name': 'first_sample_workflow',
  'outputParameters': {'data': '${get_population_data.output.response.body.data}',
                       'source': '${get_population_data.output.response.body.source}'},
  'ownerEmail': 'example@email.com',
  'restartable': True,
  'schemaVersion': 2,
  'tasks': [{'asyncComplete': False,
             'inputParameters': {'http

## DELETE /api/metadata/workflow/{name}/{version}
Removes workflow definition. It does not remove workflows associated with the definition.

In [6]:
# http://localhost:8080/swagger-ui/index.html#/workflow-resource/startWorkflow_1
url = URL_PREFIX + '/api/metadata/workflow/{name}/{version}'.format(name='first_sample_workflow', version=1)
params = {}
data = {
}

print(url)
pprint.pprint(headers)
pprint.pprint(params)

r = requests.delete(url=url, headers=headers, params=params, data=json.dumps(data))
print_response(r)

http://localhost:8080/api/metadata/workflow/first_sample_workflow/1
{'Content-Type': 'application/json'}
{}
====== Response
 200
=== >
DELETE http://localhost:8080/api/metadata/workflow/first_sample_workflow/1
{'User-Agent': 'python-requests/2.31.0', 'Accept-Encoding': 'gzip, deflate', 'Accept': '*/*', 'Connection': 'keep-alive', 'Content-Type': 'application/json', 'Content-Length': '2'}
{}
=== <
{'Content-Length': '0', 'Date': 'Mon, 08 Jul 2024 01:38:59 GMT', 'Keep-Alive': 'timeout=60', 'Connection': 'keep-alive'}
b''


## POST /api/metadata/workflow
Create a new workflow definition

In [9]:
# https://docs.conductor-oss.org/devguide/labs/first-workflow.html
url = URL_PREFIX + '/api/metadata/workflow'
params = {}
data = {
    "name": "first_sample_workflow",  # * workflow name
    "description": "First Sample Workflow",
    "version": 1,                                # workflow version
    "tasks": [  # * worker tasks
        {
            "name": "get_population_data",                     # name
            "taskReferenceName": "get_population_data",        # unique task reference name
            "inputParameters": {                               # inputs
                "http_request": {                                    # inputs of HTTP task
                    "uri": "https://datausa.io/api/data?drilldowns=Nation&measures=Population",
                    "method": "GET"
                }
            },
            "type": "HTTP"                                     # type of worker
        }
    ],
    "inputParameters": [],
    "outputParameters": {
        "data": "${get_population_data.output.response.body.data}",
        "source": "${get_population_data.output.response.body.source}"
    },
    "schemaVersion": 2,
    "restartable": True,
    "workflowStatusListenerEnabled": False,
    "ownerEmail": "example@email.com",
    "timeoutPolicy": "ALERT_ONLY",
    "timeoutSeconds": 0
}

print(url)
pprint.pprint(headers)
pprint.pprint(params)

r = requests.post(url=url, headers=headers,
                  params=params, data=json.dumps(data))
print_response(r)

http://localhost:8080/api/metadata/workflow
{'Content-Type': 'application/json'}
{}
====== Response
 200
=== >
POST http://localhost:8080/api/metadata/workflow
{'User-Agent': 'python-requests/2.31.0', 'Accept-Encoding': 'gzip, deflate', 'Accept': '*/*', 'Connection': 'keep-alive', 'Content-Type': 'application/json', 'Content-Length': '660'}
{"name": "first_sample_workflow", "description": "First Sample Workflow", "version": 1, "tasks": [{"name": "get_population_data", "taskReferenceName": "get_population_data", "inputParameters": {"http_request": {"uri": "https://datausa.io/api/data?drilldowns=Nation&measures=Population", "method": "GET"}}, "type": "HTTP"}], "inputParameters": [], "outputParameters": {"data": "${get_population_data.output.response.body.data}", "source": "${get_population_data.output.response.body.source}"}, "schemaVersion": 2, "restartable": true, "workflowStatusListenerEnabled": false, "ownerEmail": "example@email.com", "timeoutPolicy": "ALERT_ONLY", "timeoutSeconds":

## PUT /api/metadata/workflow
Create or update workflow definition

In [12]:
# https://docs.conductor-oss.org/devguide/labs/first-workflow.html
url = URL_PREFIX + '/api/metadata/workflow'
params = {}
data = [
    {
        "name": "first_sample_workflow",  # * workflow name
        "description": "First Sample Workflow",
        "version": 1,                                # workflow version
        "tasks": [  # * worker tasks
            {
                "name": "get_population_data",                     # name
                "taskReferenceName": "get_population_data",        # unique task reference name
                "inputParameters": {                               # inputs
                    "http_request": {                                    # inputs of HTTP task
                        "uri": "https://datausa.io/api/data?drilldowns=Nation&measures=Population",
                        "method": "GET"
                    }
                },
                "type": "HTTP"                                     # type of worker
            }
        ],
        "inputParameters": [],
        "outputParameters": {
            "data": "${get_population_data.output.response.body.data}",
            "source": "${get_population_data.output.response.body.source}"
        },
        "schemaVersion": 2,
        "restartable": True,
        "workflowStatusListenerEnabled": False,
        "ownerEmail": "example@email.com",
        "timeoutPolicy": "ALERT_ONLY",
        "timeoutSeconds": 0
    }
]

print(url)
pprint.pprint(headers)
pprint.pprint(params)

r = requests.put(url=url, headers=headers,
                 params=params, data=json.dumps(data))
print_response(r)

http://localhost:8080/api/metadata/workflow
{'Content-Type': 'application/json'}
{}
====== Response
 200
=== >
PUT http://localhost:8080/api/metadata/workflow
{'User-Agent': 'python-requests/2.31.0', 'Accept-Encoding': 'gzip, deflate', 'Accept': '*/*', 'Connection': 'keep-alive', 'Content-Type': 'application/json', 'Content-Length': '662'}
[{"name": "first_sample_workflow", "description": "First Sample Workflow", "version": 1, "tasks": [{"name": "get_population_data", "taskReferenceName": "get_population_data", "inputParameters": {"http_request": {"uri": "https://datausa.io/api/data?drilldowns=Nation&measures=Population", "method": "GET"}}, "type": "HTTP"}], "inputParameters": [], "outputParameters": {"data": "${get_population_data.output.response.body.data}", "source": "${get_population_data.output.response.body.source}"}, "schemaVersion": 2, "restartable": true, "workflowStatusListenerEnabled": false, "ownerEmail": "example@email.com", "timeoutPolicy": "ALERT_ONLY", "timeoutSeconds":

## Events and Event Handlers

In [10]:
# https://docs.conductor-oss.org/devguide/labs/first-workflow.html
url = URL_PREFIX + '/api/metadata/workflow'
params = {}
data = [
    {
        "name": "test_workflow_for_eventHandler",
        "description": "A test workflow to start another workflow with EventHandler",
        "version": 1,
        "tasks": [
            {
                "name": "test_start_workflow_event",
                "taskReferenceName": "start_workflow_with_event",
                "type": "EVENT",       # Event task to start another workflow
                "sink": "conductor"
            },
            {
                "name": "test_task_tobe_completed_by_eventHandler",
                "taskReferenceName": "test_task_tobe_completed_by_eventHandler",
                "type": "WAIT"        # Wait task to be completed by an event
            }
        ],
        "ownerEmail": "example@email.com",
    },

    {
        "name": "test_workflow_startedBy_eventHandler",
        "description": "A test workflow which is started by EventHandler, and then goes on to complete task in another workflow.",
        "version": 1,
        "tasks": [
            {
                "name": "test_complete_task_event",
                "taskReferenceName": "complete_task_with_event",
                "inputParameters": {
                    "sourceWorkflowId": "${workflow.input.sourceWorkflowId}"
                },
                "type": "EVENT",     # Event task to generate an event to complete the Wait task above
                "sink": "conductor"
            }
        ],
        "ownerEmail": "example@email.com",
    }
]

print(url)
pprint.pprint(headers)
pprint.pprint(params)

r = requests.put(url=url, headers=headers,
                 params=params, data=json.dumps(data))
print_response(r)

http://localhost:8080/api/metadata/workflow
{'Content-Type': 'application/json'}
{}
====== Response
 200
=== >
PUT http://localhost:8080/api/metadata/workflow
{'User-Agent': 'python-requests/2.31.0', 'Accept-Encoding': 'gzip, deflate', 'Accept': '*/*', 'Connection': 'keep-alive', 'Content-Type': 'application/json', 'Content-Length': '881'}
[{"name": "test_workflow_for_eventHandler", "description": "A test workflow to start another workflow with EventHandler", "version": 1, "tasks": [{"name": "test_start_workflow_event", "taskReferenceName": "start_workflow_with_event", "type": "EVENT", "sink": "conductor"}, {"name": "test_task_tobe_completed_by_eventHandler", "taskReferenceName": "test_task_tobe_completed_by_eventHandler", "type": "WAIT"}], "ownerEmail": "example@email.com"}, {"name": "test_workflow_startedBy_eventHandler", "description": "A test workflow which is started by EventHandler, and then goes on to complete task in another workflow.", "version": 1, "tasks": [{"name": "test_co

# Workflow

## POST /api/workflow/{name}
Start a new workflow.Returns the ID of the workflow instance that can be later used for tracking

In [18]:
# http://localhost:8080/swagger-ui/index.html#/workflow-resource/startWorkflow_1
url = URL_PREFIX + '/api/workflow/{name}'.format(name='first_sample_workflow')
params = {}
data = {
}

print(url)
pprint.pprint(headers)
pprint.pprint(params)

r = requests.post(url=url, headers=headers, params=params, data=json.dumps(data))
print_response(r)
if r.status_code == 200:
  _workflow_id = r.content.decode()
print(_workflow_id)

http://localhost:8080/api/workflow/first_sample_workflow
{'Content-Type': 'application/json'}
{}
====== Response
 200
=== >
POST http://localhost:8080/api/workflow/first_sample_workflow
{'User-Agent': 'python-requests/2.31.0', 'Accept-Encoding': 'gzip, deflate', 'Accept': '*/*', 'Connection': 'keep-alive', 'Content-Type': 'application/json', 'Content-Length': '2'}
{}
=== <
{'Content-Type': 'text/plain;charset=UTF-8', 'Content-Length': '36', 'Date': 'Mon, 08 Jul 2024 01:52:21 GMT', 'Keep-Alive': 'timeout=60', 'Connection': 'keep-alive'}
b'74522120-cb7c-48ae-8c0b-ed0d3bdcd564'
74522120-cb7c-48ae-8c0b-ed0d3bdcd564


## GET /api/workflow/{workflowId}
Gets the workflow by workflow id

In [19]:
# _workflow_id='bdc0eb03-da18-42b8-b920-982c0443ec7e'
# UI: http://localhost:5000/execution/bdc0eb03-da18-42b8-b920-982c0443ec7e
url = URL_PREFIX + '/api/workflow/{workflowId}'.format(workflowId=_workflow_id)
params = {
}
print(url)
pprint.pprint(headers)
pprint.pprint(params)

r = requests.get(url=url, headers=headers, params=params)
print_response(r)

http://localhost:8080/api/workflow/74522120-cb7c-48ae-8c0b-ed0d3bdcd564
{'Content-Type': 'application/json'}
{}
====== Response
 200
=== >
GET http://localhost:8080/api/workflow/74522120-cb7c-48ae-8c0b-ed0d3bdcd564
{'User-Agent': 'python-requests/2.31.0', 'Accept-Encoding': 'gzip, deflate', 'Accept': '*/*', 'Connection': 'keep-alive', 'Content-Type': 'application/json'}
None
=== <
{'Content-Type': 'application/json', 'Transfer-Encoding': 'chunked', 'Date': 'Mon, 08 Jul 2024 01:52:25 GMT', 'Keep-Alive': 'timeout=60', 'Connection': 'keep-alive'}
{'createTime': 1720403541485,
 'endTime': 1720403542108,
 'failedReferenceTaskNames': [],
 'failedTaskNames': [],
 'input': {},
 'lastRetriedTime': 0,
 'output': {'data': [{'ID Nation': '01000US',
                      'ID Year': 2022,
                      'Nation': 'United States',
                      'Population': 331097593,
                      'Slug Nation': 'united-states',
                      'Year': '2022'},
                     {'ID

## POST /api/workflow/{name}/correlated
Lists workflows for the given correlation id list

In [18]:
url = URL_PREFIX + '/api/workflow/{name}/correlated'.format(name='test_workflow_for_eventHandler')
params = {}
data = {
}

print(url)
pprint.pprint(headers)
pprint.pprint(params)

r = requests.post(url=url, headers=headers, params=params, data=json.dumps(data))
print_response(r)

http://localhost:8080/api/workflow/test_workflow_for_eventHandler/correlated
{'Content-Type': 'application/json'}
{}
====== Response
 500
=== >
POST http://localhost:8080/api/workflow/test_workflow_for_eventHandler/correlated
{'User-Agent': 'python-requests/2.31.0', 'Accept-Encoding': 'gzip, deflate', 'Accept': '*/*', 'Connection': 'keep-alive', 'Content-Type': 'application/json', 'Content-Length': '2'}
{}
=== <
{'Content-Type': 'application/json', 'Transfer-Encoding': 'chunked', 'Date': 'Mon, 08 Jul 2024 02:52:13 GMT', 'Connection': 'close'}
{'instance': 'ea1e8f2d944f',
 'message': 'JSON parse error: Cannot deserialize value of type '
            '`java.util.ArrayList<java.lang.String>` from Object value (token '
            '`JsonToken.START_OBJECT`)',
 'retryable': False,
 'status': 500}
2cd9e154-88b7-4223-9b93-18d3791098ac


## GET /api/workflow/search-v2
Search for workflows based on payload and other parameters

In [20]:
url = URL_PREFIX + '/api/workflow/search-v2'
params = {
  'freeText': 'test_workflow_startedBy_eventHandler',
  'query': None
}
print(url)
pprint.pprint(headers)
pprint.pprint(params)

r = requests.get(url=url, headers=headers, params=params)
print_response(r)

http://localhost:8080/api/workflow/search-v2
{'Content-Type': 'application/json'}
{'freeText': 'test_workflow_startedBy_eventHandler', 'query': None}
====== Response
 200
=== >
GET http://localhost:8080/api/workflow/search-v2?freeText=test_workflow_startedBy_eventHandler
{'User-Agent': 'python-requests/2.31.0', 'Accept-Encoding': 'gzip, deflate', 'Accept': '*/*', 'Connection': 'keep-alive', 'Content-Type': 'application/json'}
None
=== <
{'Content-Type': 'application/json', 'Transfer-Encoding': 'chunked', 'Date': 'Mon, 08 Jul 2024 02:55:39 GMT', 'Keep-Alive': 'timeout=60', 'Connection': 'keep-alive'}
{'results': [{'createTime': 1720405939382,
              'endTime': 1720405939420,
              'event': 'conductor:test_workflow_for_eventHandler:start_workflow_with_event',
              'failedReferenceTaskNames': [],
              'failedTaskNames': [],
              'input': {'conductor.event.messageId': 'd70df840-0864-4948-aac4-0566c23b3851',
                        'conductor.event.

## Events and Event Handlers

In [11]:
# start workflow
url = URL_PREFIX + '/api/workflow/{name}'.format(name='test_workflow_for_eventHandler')
params = {}
data = {
}

print(url)
pprint.pprint(headers)
pprint.pprint(params)

r = requests.post(url=url, headers=headers, params=params, data=json.dumps(data))
print_response(r)
if r.status_code == 200:
  _workflow_id = r.content.decode()
print(_workflow_id)

http://localhost:8080/api/workflow/test_workflow_for_eventHandler
{'Content-Type': 'application/json'}
{}
====== Response
 200
=== >
POST http://localhost:8080/api/workflow/test_workflow_for_eventHandler
{'User-Agent': 'python-requests/2.31.0', 'Accept-Encoding': 'gzip, deflate', 'Accept': '*/*', 'Connection': 'keep-alive', 'Content-Type': 'application/json', 'Content-Length': '2'}
{}
=== <
{'Content-Type': 'text/plain;charset=UTF-8', 'Content-Length': '36', 'Date': 'Mon, 08 Jul 2024 02:32:17 GMT', 'Keep-Alive': 'timeout=60', 'Connection': 'keep-alive'}
b'2cd9e154-88b7-4223-9b93-18d3791098ac'
2cd9e154-88b7-4223-9b93-18d3791098ac


In [14]:
# query workflow by workflowId
print("UI: {ui_url}".format(ui_url="http://localhost:5000/execution/{workflowId}".format(workflowId=_workflow_id)))
url = URL_PREFIX + '/api/workflow/{workflowId}'.format(workflowId=_workflow_id)
params = {
}
print(url)
pprint.pprint(headers)
pprint.pprint(params)

r = requests.get(url=url, headers=headers, params=params)
print_response(r)
if r.status_code == 200:
  _taskIds = [task['taskId'] for task in r.json()['tasks']]
print(_taskIds)

UI: http://localhost:5000/execution/2cd9e154-88b7-4223-9b93-18d3791098ac
http://localhost:8080/api/workflow/2cd9e154-88b7-4223-9b93-18d3791098ac
{'Content-Type': 'application/json'}
{}
====== Response
 200
=== >
GET http://localhost:8080/api/workflow/2cd9e154-88b7-4223-9b93-18d3791098ac
{'User-Agent': 'python-requests/2.31.0', 'Accept-Encoding': 'gzip, deflate', 'Accept': '*/*', 'Connection': 'keep-alive', 'Content-Type': 'application/json'}
None
=== <
{'Content-Type': 'application/json', 'Transfer-Encoding': 'chunked', 'Date': 'Mon, 08 Jul 2024 02:45:33 GMT', 'Keep-Alive': 'timeout=60', 'Connection': 'keep-alive'}
{'createTime': 1720405937194,
 'endTime': 1720405941328,
 'failedReferenceTaskNames': [],
 'failedTaskNames': [],
 'input': {},
 'lastRetriedTime': 0,
 'output': {'conductor.event.messageId': '3398c011-b8d2-47c5-ba89-38693bd1550f',
            'conductor.event.name': 'conductor:test_workflow_startedBy_eventHandler:complete_task_with_event',
            'taskId': None,
      

# Task

## GET /api/tasks/{taskId}
Get task by Id

In [15]:
url = URL_PREFIX + '/api/tasks/{taskId}'.format(taskId=_taskIds[0])
params = {
}
print(url)
pprint.pprint(headers)
pprint.pprint(params)

r = requests.get(url=url, headers=headers, params=params)
print_response(r)

http://localhost:8080/api/tasks/d70df840-0864-4948-aac4-0566c23b3851
{'Content-Type': 'application/json'}
{}
====== Response
 200
=== >
GET http://localhost:8080/api/tasks/d70df840-0864-4948-aac4-0566c23b3851
{'User-Agent': 'python-requests/2.31.0', 'Accept-Encoding': 'gzip, deflate', 'Accept': '*/*', 'Connection': 'keep-alive', 'Content-Type': 'application/json'}
None
=== <
{'Content-Type': 'application/json', 'Transfer-Encoding': 'chunked', 'Date': 'Mon, 08 Jul 2024 02:45:48 GMT', 'Keep-Alive': 'timeout=60', 'Connection': 'keep-alive'}
{'callbackAfterSeconds': 0,
 'callbackFromWorker': True,
 'endTime': 1720405937255,
 'executed': True,
 'inputData': {'asyncComplete': False, 'sink': 'conductor'},
 'iteration': 0,
 'loopOverTask': False,
 'outputData': {'asyncComplete': False,
                'correlationId': None,
                'event_produced': 'conductor:test_workflow_for_eventHandler:start_workflow_with_event',
                'sink': 'conductor',
                'workflowInstan

## GET /api/tasks/{taskId}/log
Get Task Execution Logs

In [16]:
url = URL_PREFIX + '/api/tasks/{taskId}/log'.format(taskId=_taskIds[0])
params = {
}
print(url)
pprint.pprint(headers)
pprint.pprint(params)

r = requests.get(url=url, headers=headers, params=params)
print_response(r)

http://localhost:8080/api/tasks/d70df840-0864-4948-aac4-0566c23b3851/log
{'Content-Type': 'application/json'}
{}
====== Response
 200
=== >
GET http://localhost:8080/api/tasks/d70df840-0864-4948-aac4-0566c23b3851/log
{'User-Agent': 'python-requests/2.31.0', 'Accept-Encoding': 'gzip, deflate', 'Accept': '*/*', 'Connection': 'keep-alive', 'Content-Type': 'application/json'}
None
=== <
{'Content-Length': '0', 'Date': 'Mon, 08 Jul 2024 02:46:18 GMT', 'Keep-Alive': 'timeout=60', 'Connection': 'keep-alive'}
b''


## Events and Event Handlers

In [17]:
for _taskId in _taskIds:
  url = URL_PREFIX + '/api/tasks/{taskId}'.format(taskId=_taskId)
  params = {
  }
  print(url)
  pprint.pprint(headers)
  pprint.pprint(params)

  r = requests.get(url=url, headers=headers, params=params)
  print_response(r)

http://localhost:8080/api/tasks/d70df840-0864-4948-aac4-0566c23b3851
{'Content-Type': 'application/json'}
{}
====== Response
 200
=== >
GET http://localhost:8080/api/tasks/d70df840-0864-4948-aac4-0566c23b3851
{'User-Agent': 'python-requests/2.31.0', 'Accept-Encoding': 'gzip, deflate', 'Accept': '*/*', 'Connection': 'keep-alive', 'Content-Type': 'application/json'}
None
=== <
{'Content-Type': 'application/json', 'Transfer-Encoding': 'chunked', 'Date': 'Mon, 08 Jul 2024 02:47:10 GMT', 'Keep-Alive': 'timeout=60', 'Connection': 'keep-alive'}
{'callbackAfterSeconds': 0,
 'callbackFromWorker': True,
 'endTime': 1720405937255,
 'executed': True,
 'inputData': {'asyncComplete': False, 'sink': 'conductor'},
 'iteration': 0,
 'loopOverTask': False,
 'outputData': {'asyncComplete': False,
                'correlationId': None,
                'event_produced': 'conductor:test_workflow_for_eventHandler:start_workflow_with_event',
                'sink': 'conductor',
                'workflowInstan

# Task Domains

# Event

## GET /api/event
Get all the event handlers

In [13]:
url = URL_PREFIX + '/api/event'
params = {}

print(url)
pprint.pprint(headers)
pprint.pprint(params)

r = requests.get(url=url, headers=headers, params=params)
print_response(r)

http://localhost:8080/api/event
{'Content-Type': 'application/json'}
{}
====== Response
 200
=== >
GET http://localhost:8080/api/event
{'User-Agent': 'python-requests/2.31.0', 'Accept-Encoding': 'gzip, deflate', 'Accept': '*/*', 'Connection': 'keep-alive', 'Content-Type': 'application/json'}
None
=== <
{'Content-Type': 'application/json', 'Transfer-Encoding': 'chunked', 'Date': 'Mon, 08 Jul 2024 02:33:48 GMT', 'Keep-Alive': 'timeout=60', 'Connection': 'keep-alive'}
[{'actions': [{'action': 'start_workflow',
               'expandInlineJSON': False,
               'start_workflow': {'input': {'sourceWorkflowId': '${workflowInstanceId}'},
                                  'name': 'test_workflow_startedBy_eventHandler'}}],
  'active': True,
  'event': 'conductor:test_workflow_for_eventHandler:start_workflow_with_event',
  'name': 'test_start_workflow'},
 {'actions': [{'action': 'complete_task',
               'complete_task': {'output': {},
                                 'taskRefName': 

## POST /api/event
Add a new event handler.

In [4]:
url = URL_PREFIX + '/api/event'
params = {}
# Event handlers: https://docs.conductor-oss.org/documentation/configuration/eventhandlers.html
# supported actions:
## start a workflow
## fail a task
## complete a task

# Input for starting a workflow and output when completing / failing task 
# follows the same expressions used for wiring task inputs.

data = {
    "name": "test_start_workflow",
    # format for Conductor queue: conductor:{workflow_name}:{taskReferenceName}
    "event": "conductor:test_workflow_for_eventHandler:start_workflow_with_event",
    "actions": [
        {                                     # start workflow action
            "action": "start_workflow",
            "start_workflow": {               # https://docs.conductor-oss.org/documentation/api/startworkflow.html
                "name": "test_workflow_startedBy_eventHandler",
                "input": {
                    "sourceWorkflowId": "${workflowInstanceId}"
                }
            }
        }
    ],
    "active": True
}

print(url)
pprint.pprint(headers)
pprint.pprint(params)

r = requests.post(url=url, headers=headers,
                  params=params, data=json.dumps(data))
print_response(r)

http://localhost:8080/api/event
{'Content-Type': 'application/json'}
{}
====== Response
 200
=== >
POST http://localhost:8080/api/event
{'User-Agent': 'python-requests/2.31.0', 'Accept-Encoding': 'gzip, deflate', 'Accept': '*/*', 'Connection': 'keep-alive', 'Content-Type': 'application/json', 'Content-Length': '293'}
{"name": "test_start_workflow", "event": "conductor:test_workflow_for_eventHandler:start_workflow_with_event", "actions": [{"action": "start_workflow", "start_workflow": {"name": "test_workflow_startedBy_eventHandler", "input": {"sourceWorkflowId": "${workflowInstanceId}"}}}], "active": true}
=== <
{'Content-Length': '0', 'Date': 'Mon, 08 Jul 2024 02:25:50 GMT', 'Keep-Alive': 'timeout=60', 'Connection': 'keep-alive'}
b''


In [5]:
url = URL_PREFIX + '/api/event'
params = {}
data = {
    "name": "test_complete_task_event",
    "event": "conductor:test_workflow_startedBy_eventHandler:complete_task_with_event",
    "actions": [
        {
            "action": "complete_task",
            "complete_task": {               # complete task action
                "workflowId": "${sourceWorkflowId}",
                "taskRefName": "test_task_tobe_completed_by_eventHandler"
            }
        }
    ],
    "active": True
}

print(url)
pprint.pprint(headers)
pprint.pprint(params)

r = requests.post(url=url, headers=headers,
                  params=params, data=json.dumps(data))
print_response(r)

http://localhost:8080/api/event
{'Content-Type': 'application/json'}
{}
====== Response
 200
=== >
POST http://localhost:8080/api/event
{'User-Agent': 'python-requests/2.31.0', 'Accept-Encoding': 'gzip, deflate', 'Accept': '*/*', 'Connection': 'keep-alive', 'Content-Type': 'application/json', 'Content-Length': '293'}
{"name": "test_complete_task_event", "event": "conductor:test_workflow_startedBy_eventHandler:complete_task_with_event", "actions": [{"action": "complete_task", "complete_task": {"workflowId": "${sourceWorkflowId}", "taskRefName": "test_task_tobe_completed_by_eventHandler"}}], "active": true}
=== <
{'Content-Length': '0', 'Date': 'Mon, 08 Jul 2024 02:25:53 GMT', 'Keep-Alive': 'timeout=60', 'Connection': 'keep-alive'}
b''


# Kitchen Sink

## Create workflow definition

In [21]:
url = URL_PREFIX + '/api/metadata/workflow'
params = {}
data = {
    "name": "kitchensink",
    "description": "kitchensink workflow",
    "version": 1,
    "tasks": [
        {
            "name": "task_1",
            "taskReferenceName": "task_1",
            "inputParameters": {
                "mod": "${workflow.input.mod}",
                "oddEven": "${workflow.input.oddEven}"
            },
            "type": "SIMPLE"
        },
        {
            "name": "event_task",
            "taskReferenceName": "event_0",
            "inputParameters": {
                "mod": "${workflow.input.mod}",
                "oddEven": "${workflow.input.oddEven}"
            },
            "type": "EVENT",
            "sink": "conductor"
        },
        {
            "name": "dyntask",
            "taskReferenceName": "task_2",
            "inputParameters": {
                "taskToExecute": "${workflow.input.task2Name}"
            },
            "type": "DYNAMIC",
            "dynamicTaskNameParam": "taskToExecute"
        },
        {
            "name": "oddEvenDecision",
            "taskReferenceName": "oddEvenDecision",
            "inputParameters": {
                "oddEven": "${task_2.output.oddEven}"
            },
            "type": "DECISION",
            "caseValueParam": "oddEven",
            "decisionCases": {
                "0": [
                    {
                        "name": "task_4",
                        "taskReferenceName": "task_4",
                        "inputParameters": {
                            "mod": "${task_2.output.mod}",
                            "oddEven": "${task_2.output.oddEven}"
                        },
                        "type": "SIMPLE"
                    },
                    {
                        "name": "dynamic_fanout",
                        "taskReferenceName": "fanout1",
                        "inputParameters": {
                            "dynamicTasks": "${task_4.output.dynamicTasks}",
                            "input": "${task_4.output.inputs}"
                        },
                        "type": "FORK_JOIN_DYNAMIC",
                        "dynamicForkTasksParam": "dynamicTasks",
                        "dynamicForkTasksInputParamName": "input"
                    },
                    {
                        "name": "dynamic_join",
                        "taskReferenceName": "join1",
                        "type": "JOIN"
                    }
                ],
                "1": [
                    {
                        "name": "fork_join",
                        "taskReferenceName": "forkx",
                        "type": "FORK_JOIN",
                        "forkTasks": [
                            [
                                {
                                    "name": "task_10",
                                    "taskReferenceName": "task_10",
                                    "type": "SIMPLE"
                                },
                                {
                                    "name": "sub_workflow_x",
                                    "taskReferenceName": "wf3",
                                    "inputParameters": {
                                        "mod": "${task_1.output.mod}",
                                        "oddEven": "${task_1.output.oddEven}"
                                    },
                                    "type": "SUB_WORKFLOW",
                                    "subWorkflowParam": {
                                        "name": "sub_flow_1",
                                        "version": 1
                                    }
                                }
                            ],
                            [
                                {
                                    "name": "task_11",
                                    "taskReferenceName": "task_11",
                                    "type": "SIMPLE"
                                },
                                {
                                    "name": "sub_workflow_x",
                                    "taskReferenceName": "wf4",
                                    "inputParameters": {
                                        "mod": "${task_1.output.mod}",
                                        "oddEven": "${task_1.output.oddEven}"
                                    },
                                    "type": "SUB_WORKFLOW",
                                    "subWorkflowParam": {
                                        "name": "sub_flow_1",
                                        "version": 1
                                    }
                                }
                            ]
                        ]
                    },
                    {
                        "name": "join",
                        "taskReferenceName": "join2",
                        "type": "JOIN",
                        "joinOn": [
                            "wf3",
                            "wf4"
                        ]
                    }
                ]
            }
        },
        {
            "name": "search_elasticsearch",
            "taskReferenceName": "get_es_1",
            "inputParameters": {
                "http_request": {
                    "uri": "http://localhost:9200/conductor/_search?size=10",
                    "method": "GET"
                }
            },
            "type": "HTTP"
        },
        {
            "name": "task_30",
            "taskReferenceName": "task_30",
            "inputParameters": {
                "statuses": "${get_es_1.output..status}",
                "workflowIds": "${get_es_1.output..workflowId}"
            },
            "type": "SIMPLE"
        }
    ],
    "outputParameters": {
        "statues": "${get_es_1.output..status}",
        "workflowIds": "${get_es_1.output..workflowId}"
    },
    "ownerEmail": "example@email.com",
    "schemaVersion": 2
}

print(url)
pprint.pprint(headers)
pprint.pprint(params)

r = requests.post(url=url, headers=headers,
                  params=params, data=json.dumps(data))
print_response(r)

http://localhost:8080/api/metadata/workflow
{'Content-Type': 'application/json'}
{}
====== Response
 200
=== >
POST http://localhost:8080/api/metadata/workflow
{'User-Agent': 'python-requests/2.31.0', 'Accept-Encoding': 'gzip, deflate', 'Accept': '*/*', 'Connection': 'keep-alive', 'Content-Type': 'application/json', 'Content-Length': '2669'}
{"name": "kitchensink", "description": "kitchensink workflow", "version": 1, "tasks": [{"name": "tas...
=== <
{'Content-Length': '0', 'Date': 'Mon, 08 Jul 2024 03:08:39 GMT', 'Keep-Alive': 'timeout=60', 'Connection': 'keep-alive'}
b''


## Start workflow execution

In [22]:
url = URL_PREFIX + '/api/workflow/{name}'.format(name='kitchensink')
params = {}
data = {
    "task2Name": "task_5"
}

print(url)
pprint.pprint(headers)
pprint.pprint(params)

r = requests.post(url=url, headers=headers,
                  params=params, data=json.dumps(data))
print_response(r)
if r.status_code == 200:
    _workflow_id = r.content.decode()
print(_workflow_id)

http://localhost:8080/api/workflow/kitchensink
{'Content-Type': 'application/json'}
{}
====== Response
 200
=== >
POST http://localhost:8080/api/workflow/kitchensink
{'User-Agent': 'python-requests/2.31.0', 'Accept-Encoding': 'gzip, deflate', 'Accept': '*/*', 'Connection': 'keep-alive', 'Content-Type': 'application/json', 'Content-Length': '23'}
{"task2Name": "task_5"}
=== <
{'Content-Type': 'text/plain;charset=UTF-8', 'Content-Length': '36', 'Date': 'Mon, 08 Jul 2024 03:11:33 GMT', 'Keep-Alive': 'timeout=60', 'Connection': 'keep-alive'}
b'c29fb3dd-7e64-484a-b987-643573acc7ac'
c29fb3dd-7e64-484a-b987-643573acc7ac


## Poll for task

In [42]:
url = URL_PREFIX + '/api/tasks/poll/{taskType}'.format(taskType='task_1')
params = {}

print(url)
pprint.pprint(headers)
pprint.pprint(params)

r = requests.get(url=url, headers=headers, params=params)
print_response(r)
if r.status_code == 200:
  _taskId = r.json()['taskId']

http://localhost:8080/api/tasks/poll/task_1
{'Content-Type': 'application/json'}
{}
====== Response
 204
=== >
GET http://localhost:8080/api/tasks/poll/task_1
{'User-Agent': 'python-requests/2.31.0', 'Accept-Encoding': 'gzip, deflate', 'Accept': '*/*', 'Connection': 'keep-alive', 'Content-Type': 'application/json'}
None
=== <
{'Date': 'Mon, 08 Jul 2024 04:39:35 GMT', 'Keep-Alive': 'timeout=60', 'Connection': 'keep-alive'}
b''


## Update task status

In [43]:
_workflow_id = '8e54db6d-da83-4c63-927b-12c352e38f8d'
# _taskId = 'a604c7f7-2bbb-4989-bd26-9fb439719846'
url = URL_PREFIX + '/api/tasks'
params = {}
data = {
    "taskId": _taskId,                        #
    "workflowInstanceId": _workflow_id,       #
    "status": "COMPLETED",
    "outputData": {
        "mod": 5,
        "taskToExecute": "task_1",
        "oddEven": 0,
        "dynamicTasks": [
            {
                "name": "task_1",
                "taskReferenceName": "task_1_1",
                "type": "SIMPLE"
            },
            {
                "name": "sub_workflow_4",
                "taskReferenceName": "wf_dyn",
                "type": "SUB_WORKFLOW",
                "subWorkflowParam": {
                    "name": "sub_flow_1"
                }
            }
        ],
        "inputs": {
            "task_1_1": {},
            "wf_dyn": {}
        }
    }
}

print(url)
pprint.pprint(headers)
pprint.pprint(params)

r = requests.post(url=url, headers=headers,
                  params=params, data=json.dumps(data))
print_response(r)

http://localhost:8080/api/tasks
{'Content-Type': 'application/json'}
{}
====== Response
 200
=== >
POST http://localhost:8080/api/tasks
{'User-Agent': 'python-requests/2.31.0', 'Accept-Encoding': 'gzip, deflate', 'Accept': '*/*', 'Connection': 'keep-alive', 'Content-Type': 'application/json', 'Content-Length': '460'}
{"taskId": "8c74ca0a-1a1e-4294-80fc-528ea1c10160", "workflowInstanceId": "8e54db6d-da83-4c63-927b-12c352e38f8d", "status": "COMPLETED", "outputData": {"mod": 5, "taskToExecute": "task_1", "oddEven": 0, "dynamicTasks": [{"name": "task_1", "taskReferenceName": "task_1_1", "type": "SIMPLE"}, {"name": "sub_workflow_4", "taskReferenceName": "wf_dyn", "type": "SUB_WORKFLOW", "subWorkflowParam": {"name": "sub_flow_1"}}], "inputs": {"task_1_1": {}, "wf_dyn": {}}}}
=== <
{'Content-Type': 'text/plain;charset=UTF-8', 'Content-Length': '36', 'Date': 'Mon, 08 Jul 2024 04:39:36 GMT', 'Keep-Alive': 'timeout=60', 'Connection': 'keep-alive'}
b'8c74ca0a-1a1e-4294-80fc-528ea1c10160'


## Run com.netflix.conductor.client.sample.Main

## task_10

In [49]:
url = URL_PREFIX + '/api/tasks/poll/{taskType}'.format(taskType='task_10')
params = {}

print(url)
pprint.pprint(headers)
pprint.pprint(params)

r = requests.get(url=url, headers=headers, params=params)
print_response(r)
if r.status_code == 200:
  _taskId = r.json()['taskId']

http://localhost:8080/api/tasks/poll/task_10
{'Content-Type': 'application/json'}
{}
====== Response
 200
=== >
GET http://localhost:8080/api/tasks/poll/task_10
{'User-Agent': 'python-requests/2.31.0', 'Accept-Encoding': 'gzip, deflate', 'Accept': '*/*', 'Connection': 'keep-alive', 'Content-Type': 'application/json'}
None
=== <
{'Content-Type': 'application/json', 'Transfer-Encoding': 'chunked', 'Date': 'Mon, 08 Jul 2024 04:42:09 GMT', 'Keep-Alive': 'timeout=60', 'Connection': 'keep-alive'}
{'callbackAfterSeconds': 0,
 'callbackFromWorker': True,
 'endTime': 0,
 'executed': False,
 'inputData': {},
 'iteration': 0,
 'loopOverTask': False,
 'outputData': {},
 'pollCount': 1,
 'queueWaitTime': 206076,
 'rateLimitFrequencyInSeconds': 1,
 'rateLimitPerFrequency': 0,
 'referenceTaskName': 'task_10',
 'responseTimeoutSeconds': 3600,
 'retried': False,
 'retryCount': 0,
 'scheduledTime': 1720413523881,
 'seq': 6,
 'startDelayInSeconds': 0,
 'startTime': 1720413729957,
 'status': 'IN_PROGRESS'

In [50]:
url = URL_PREFIX + '/api/tasks'
params = {}
data = {
    "taskId": _taskId,                        #
    "workflowInstanceId": _workflow_id,       #
    "status": "COMPLETED",
    "outputData": {}
}

print(url)
pprint.pprint(headers)
pprint.pprint(params)

r = requests.post(url=url, headers=headers,
                  params=params, data=json.dumps(data))
print_response(r)

http://localhost:8080/api/tasks
{'Content-Type': 'application/json'}
{}
====== Response
 200
=== >
POST http://localhost:8080/api/tasks
{'User-Agent': 'python-requests/2.31.0', 'Accept-Encoding': 'gzip, deflate', 'Accept': '*/*', 'Connection': 'keep-alive', 'Content-Type': 'application/json', 'Content-Length': '153'}
{"taskId": "3846444e-ee43-4bf4-8463-0d8e39f83e10", "workflowInstanceId": "8e54db6d-da83-4c63-927b-12c352e38f8d", "status": "COMPLETED", "outputData": {}}
=== <
{'Content-Type': 'text/plain;charset=UTF-8', 'Content-Length': '36', 'Date': 'Mon, 08 Jul 2024 04:42:15 GMT', 'Keep-Alive': 'timeout=60', 'Connection': 'keep-alive'}
b'3846444e-ee43-4bf4-8463-0d8e39f83e10'


## task_11

In [51]:
url = URL_PREFIX + '/api/tasks/poll/{taskType}'.format(taskType='task_11')
params = {}

print(url)
pprint.pprint(headers)
pprint.pprint(params)

r = requests.get(url=url, headers=headers, params=params)
print_response(r)
if r.status_code == 200:
  _taskId = r.json()['taskId']

http://localhost:8080/api/tasks/poll/task_11
{'Content-Type': 'application/json'}
{}
====== Response
 200
=== >
GET http://localhost:8080/api/tasks/poll/task_11
{'User-Agent': 'python-requests/2.31.0', 'Accept-Encoding': 'gzip, deflate', 'Accept': '*/*', 'Connection': 'keep-alive', 'Content-Type': 'application/json'}
None
=== <
{'Content-Type': 'application/json', 'Transfer-Encoding': 'chunked', 'Date': 'Mon, 08 Jul 2024 04:42:21 GMT', 'Keep-Alive': 'timeout=60', 'Connection': 'keep-alive'}
{'callbackAfterSeconds': 0,
 'callbackFromWorker': True,
 'endTime': 0,
 'executed': False,
 'inputData': {},
 'iteration': 0,
 'loopOverTask': False,
 'outputData': {},
 'pollCount': 1,
 'queueWaitTime': 217976,
 'rateLimitFrequencyInSeconds': 1,
 'rateLimitPerFrequency': 0,
 'referenceTaskName': 'task_11',
 'responseTimeoutSeconds': 3600,
 'retried': False,
 'retryCount': 0,
 'scheduledTime': 1720413523883,
 'seq': 7,
 'startDelayInSeconds': 0,
 'startTime': 1720413741859,
 'status': 'IN_PROGRESS'

In [52]:
url = URL_PREFIX + '/api/tasks'
params = {}
data = {
    "taskId": _taskId,                        #
    "workflowInstanceId": _workflow_id,       #
    "status": "COMPLETED",
    "outputData": {}
}

print(url)
pprint.pprint(headers)
pprint.pprint(params)

r = requests.post(url=url, headers=headers,
                  params=params, data=json.dumps(data))
print_response(r)

http://localhost:8080/api/tasks
{'Content-Type': 'application/json'}
{}
====== Response
 200
=== >
POST http://localhost:8080/api/tasks
{'User-Agent': 'python-requests/2.31.0', 'Accept-Encoding': 'gzip, deflate', 'Accept': '*/*', 'Connection': 'keep-alive', 'Content-Type': 'application/json', 'Content-Length': '153'}
{"taskId": "4fc677d7-59b7-41b5-bb94-b330c3d69813", "workflowInstanceId": "8e54db6d-da83-4c63-927b-12c352e38f8d", "status": "COMPLETED", "outputData": {}}
=== <
{'Content-Type': 'text/plain;charset=UTF-8', 'Content-Length': '36', 'Date': 'Mon, 08 Jul 2024 04:42:25 GMT', 'Keep-Alive': 'timeout=60', 'Connection': 'keep-alive'}
b'4fc677d7-59b7-41b5-bb94-b330c3d69813'


## wf3

In [53]:
url = URL_PREFIX + '/api/tasks/poll/{taskType}'.format(taskType='sub_workflow_x')
params = {}

print(url)
pprint.pprint(headers)
pprint.pprint(params)

r = requests.get(url=url, headers=headers, params=params)
print_response(r)
if r.status_code == 200:
  _taskId = r.json()['taskId']

url = URL_PREFIX + '/api/tasks'
params = {}
data = {
    "taskId": _taskId,                        #
    "workflowInstanceId": _workflow_id,       #
    "status": "COMPLETED",
    "outputData": {}
}

print(url)
pprint.pprint(headers)
pprint.pprint(params)

r = requests.post(url=url, headers=headers,
                  params=params, data=json.dumps(data))
print_response(r)

http://localhost:8080/api/tasks/poll/sub_workflow_x
{'Content-Type': 'application/json'}
{}
====== Response
 204
=== >
GET http://localhost:8080/api/tasks/poll/sub_workflow_x
{'User-Agent': 'python-requests/2.31.0', 'Accept-Encoding': 'gzip, deflate', 'Accept': '*/*', 'Connection': 'keep-alive', 'Content-Type': 'application/json'}
None
=== <
{'Date': 'Mon, 08 Jul 2024 04:42:29 GMT', 'Keep-Alive': 'timeout=60', 'Connection': 'keep-alive'}
b''
http://localhost:8080/api/tasks
{'Content-Type': 'application/json'}
{}
====== Response
 200
=== >
POST http://localhost:8080/api/tasks
{'User-Agent': 'python-requests/2.31.0', 'Accept-Encoding': 'gzip, deflate', 'Accept': '*/*', 'Connection': 'keep-alive', 'Content-Type': 'application/json', 'Content-Length': '153'}
{"taskId": "4fc677d7-59b7-41b5-bb94-b330c3d69813", "workflowInstanceId": "8e54db6d-da83-4c63-927b-12c352e38f8d", "status": "COMPLETED", "outputData": {}}
=== <
{'Content-Type': 'text/plain;charset=UTF-8', 'Content-Length': '36', 'Date'

## task_6

In [55]:
url = URL_PREFIX + '/api/tasks/poll/{taskType}'.format(taskType='task_6')
params = {}

print(url)
pprint.pprint(headers)
pprint.pprint(params)

r = requests.get(url=url, headers=headers, params=params)
print_response(r)
if r.status_code == 200:
  _taskId = r.json()['taskId']

url = URL_PREFIX + '/api/tasks'
params = {}
data = {
    "taskId": _taskId,                        #
    "workflowInstanceId": _workflow_id,       #
    "status": "COMPLETED",
    "outputData": {}
}

print(url)
pprint.pprint(headers)
pprint.pprint(params)

r = requests.post(url=url, headers=headers,
                  params=params, data=json.dumps(data))
print_response(r)

http://localhost:8080/api/tasks/poll/task_6
{'Content-Type': 'application/json'}
{}
====== Response
 200
=== >
GET http://localhost:8080/api/tasks/poll/task_6
{'User-Agent': 'python-requests/2.31.0', 'Accept-Encoding': 'gzip, deflate', 'Accept': '*/*', 'Connection': 'keep-alive', 'Content-Type': 'application/json'}
None
=== <
{'Content-Type': 'application/json', 'Transfer-Encoding': 'chunked', 'Date': 'Mon, 08 Jul 2024 04:42:50 GMT', 'Keep-Alive': 'timeout=60', 'Connection': 'keep-alive'}
{'callbackAfterSeconds': 0,
 'callbackFromWorker': True,
 'endTime': 0,
 'executed': False,
 'inputData': {},
 'iteration': 0,
 'loopOverTask': False,
 'outputData': {},
 'pollCount': 1,
 'queueWaitTime': 154046,
 'rateLimitFrequencyInSeconds': 1,
 'rateLimitPerFrequency': 0,
 'referenceTaskName': 'task_6',
 'responseTimeoutSeconds': 3600,
 'retried': False,
 'retryCount': 0,
 'scheduledTime': 1720413616858,
 'seq': 2,
 'startDelayInSeconds': 0,
 'startTime': 1720413770904,
 'status': 'IN_PROGRESS',
 

## wf4

In [40]:
url = URL_PREFIX + '/api/tasks/poll/{taskType}'.format(taskType='sub_workflow_x')
params = {}

print(url)
pprint.pprint(headers)
pprint.pprint(params)

r = requests.get(url=url, headers=headers, params=params)
print_response(r)
if r.status_code == 200:
  _taskId = r.json()['taskId']

url = URL_PREFIX + '/api/tasks'
params = {}
data = {
    "taskId": _taskId,                        #
    "workflowInstanceId": _workflow_id,       #
    "status": "COMPLETED",
    "outputData": {}
}

print(url)
pprint.pprint(headers)
pprint.pprint(params)

r = requests.post(url=url, headers=headers,
                  params=params, data=json.dumps(data))
print_response(r)

http://localhost:8080/api/tasks/poll/sub_workflow_x
{'Content-Type': 'application/json'}
{}
====== Response
 204
=== >
GET http://localhost:8080/api/tasks/poll/sub_workflow_x
{'User-Agent': 'python-requests/2.31.0', 'Accept-Encoding': 'gzip, deflate', 'Accept': '*/*', 'Connection': 'keep-alive', 'Content-Type': 'application/json'}
None
=== <
{'Date': 'Mon, 08 Jul 2024 04:29:43 GMT', 'Keep-Alive': 'timeout=60', 'Connection': 'keep-alive'}
b''
http://localhost:8080/api/tasks
{'Content-Type': 'application/json'}
{}
====== Response
 200
=== >
POST http://localhost:8080/api/tasks
{'User-Agent': 'python-requests/2.31.0', 'Accept-Encoding': 'gzip, deflate', 'Accept': '*/*', 'Connection': 'keep-alive', 'Content-Type': 'application/json', 'Content-Length': '153'}
{"taskId": "b1ba954a-5dc4-42cc-bfa9-24d94d5b4b6b", "workflowInstanceId": "8e54db6d-da83-4c63-927b-12c352e38f8d", "status": "COMPLETED", "outputData": {}}
=== <
{'Content-Type': 'text/plain;charset=UTF-8', 'Content-Length': '36', 'Date'

## task_6

In [57]:
url = URL_PREFIX + '/api/tasks/poll/{taskType}'.format(taskType='task_6')
params = {}

print(url)
pprint.pprint(headers)
pprint.pprint(params)

r = requests.get(url=url, headers=headers, params=params)
print_response(r)
if r.status_code == 200:
  _taskId = r.json()['taskId']

url = URL_PREFIX + '/api/tasks'
params = {}
data = {
    "taskId": _taskId,                        #
    "workflowInstanceId": _workflow_id,       #
    "status": "COMPLETED",
    "outputData": {}
}

print(url)
pprint.pprint(headers)
pprint.pprint(params)

r = requests.post(url=url, headers=headers,
                  params=params, data=json.dumps(data))
print_response(r)

http://localhost:8080/api/tasks/poll/task_6
{'Content-Type': 'application/json'}
{}
====== Response
 200
=== >
GET http://localhost:8080/api/tasks/poll/task_6
{'User-Agent': 'python-requests/2.31.0', 'Accept-Encoding': 'gzip, deflate', 'Accept': '*/*', 'Connection': 'keep-alive', 'Content-Type': 'application/json'}
None
=== <
{'Content-Type': 'application/json', 'Transfer-Encoding': 'chunked', 'Date': 'Mon, 08 Jul 2024 04:43:42 GMT', 'Keep-Alive': 'timeout=60', 'Connection': 'keep-alive'}
{'callbackAfterSeconds': 0,
 'callbackFromWorker': True,
 'endTime': 0,
 'executed': False,
 'inputData': {},
 'iteration': 0,
 'loopOverTask': False,
 'outputData': {},
 'pollCount': 1,
 'queueWaitTime': 75917,
 'rateLimitFrequencyInSeconds': 1,
 'rateLimitPerFrequency': 0,
 'referenceTaskName': 'task_6',
 'responseTimeoutSeconds': 3600,
 'retried': False,
 'retryCount': 0,
 'scheduledTime': 1720413746210,
 'seq': 2,
 'startDelayInSeconds': 0,
 'startTime': 1720413822127,
 'status': 'IN_PROGRESS',
 '

## task_30

In [58]:
url = URL_PREFIX + '/api/tasks/poll/{taskType}'.format(taskType='task_30')
params = {}

print(url)
pprint.pprint(headers)
pprint.pprint(params)

r = requests.get(url=url, headers=headers, params=params)
print_response(r)
if r.status_code == 200:
  _taskId = r.json()['taskId']

url = URL_PREFIX + '/api/tasks'
params = {}
data = {
    "taskId": _taskId,                        #
    "workflowInstanceId": _workflow_id,       #
    "status": "COMPLETED",
    "outputData": {}
}

print(url)
pprint.pprint(headers)
pprint.pprint(params)

r = requests.post(url=url, headers=headers,
                  params=params, data=json.dumps(data))
print_response(r)

http://localhost:8080/api/tasks/poll/task_30
{'Content-Type': 'application/json'}
{}
====== Response
 200
=== >
GET http://localhost:8080/api/tasks/poll/task_30
{'User-Agent': 'python-requests/2.31.0', 'Accept-Encoding': 'gzip, deflate', 'Accept': '*/*', 'Connection': 'keep-alive', 'Content-Type': 'application/json'}
None
=== <
{'Content-Type': 'application/json', 'Transfer-Encoding': 'chunked', 'Date': 'Mon, 08 Jul 2024 04:48:46 GMT', 'Keep-Alive': 'timeout=60', 'Connection': 'keep-alive'}
{'callbackAfterSeconds': 0,
 'callbackFromWorker': True,
 'endTime': 0,
 'executed': False,
 'inputData': {'statuses': ['COMPLETED',
                            'COMPLETED',
                            'COMPLETED',
                            'COMPLETED',
                            'COMPLETED',
                            'RUNNING',
                            'COMPLETED',
                            'COMPLETED',
                            'COMPLETED',
                            'COMPLETED'],
   

## Delete workflow definition

In [29]:
url = URL_PREFIX + '/api/metadata/workflow/{name}/{version}'.format(name='kitchensink', version=1)
params = {}
data = {
}

print(url)
pprint.pprint(headers)
pprint.pprint(params)

r = requests.delete(url=url, headers=headers, params=params, data=json.dumps(data))
print_response(r)

http://localhost:8080/api/metadata/workflow/kitchensink/1
{'Content-Type': 'application/json'}
{}
====== Response
 200
=== >
DELETE http://localhost:8080/api/metadata/workflow/kitchensink/1
{'User-Agent': 'python-requests/2.31.0', 'Accept-Encoding': 'gzip, deflate', 'Accept': '*/*', 'Connection': 'keep-alive', 'Content-Type': 'application/json', 'Content-Length': '2'}
{}
=== <
{'Content-Length': '0', 'Date': 'Mon, 08 Jul 2024 03:40:31 GMT', 'Keep-Alive': 'timeout=60', 'Connection': 'keep-alive'}
b''
